In [4]:
'''
个股倍量追踪
'''
import sys
sys.path.append('/Users/flee/myspace/utils')
import fIO
import requests
import json 
from pandas import DataFrame
import pandas as pd
import numpy as np
import time
import threading
import fmail
import threading


## 全局变量
result = []
last_result = []

## 5分钟价格：[open, close, high, low], 东财5分钟成交量在每日9.35的值未计算开盘竞价的价格变动与成交量
DAILY_DIVISION_COUNT = 48
LAST_INDEX = 1439
VOL_COLUMN_INDEX = 3

# 判断传入的时间是否早于当前时刻mincount分钟，若过早超出门限则为无效数据无需处理
def isEffectiveTime(lastDateStr, mincount):
    lastTime = time.strptime(lastDateStr, "%Y-%m-%d %H:%M")
    lastTimeStamp = int(time.mktime(lastTime))
    currentTimeStamp = int(time.time())
    accept_diff = mincount * 60
    return currentTimeStamp - lastTimeStamp < accept_diff

## 获取东财请求所需股票代码
def getSuffixCode(code):
    if code[0] == '6':
        return code + '1'
    else: 
        return code + '2'


## 获取列表平均值
def getMean(vols):
    return np.mean(np.array(vols))


## 获取中位数
def getMedian(vols):
    return np.median(np.array(vols))


## 判断是否成交倍量
def isDoubleVol(vol, mean_vol):
    if vol > 2 * mean_vol:
        return True
    else:
        return False
    
## 判断股票是否满足倍量要求
def isTargetStock(code_id):
    easy_url = 'http://pdfm.eastmoney.com/EM_UBG_PDTI_Fast/api/js?rtntype=5&token=4f1862fc3b5e77c150a2b985b12db0fd&type=m5k&authorityType=fa&id='
    final_url = easy_url + getSuffixCode(code_id)
    response = requests.get(final_url)
    data_str = response.text.split('(')[-1].split(')')[0]
    data = json.loads(data_str)['data']
    if not isEffectiveTime(data[-2].split(',')[0], 11):
        return False    
    time_list = []
    vol_list = []
    for stock_tick in data:
        tick_array = stock_tick.split(',')
        time_list.append(tick_array[0])
        vol_list.append(int(tick_array[6])/1000)
    mean_vol = max(getMean(vol_list), getMedian(vol_list))
    return isDoubleVol(vol_list[-1], mean_vol) or isDoubleVol(vol_list[-2], mean_vol)  


## 遍历codes中的目标股票是否满足要求
def fetchResult(codes):
    result_codes = []
    failed_codes = []
    start = time.time()
    for code in codes:
        try:
        if isTargetStock(code):
            result_codes.append(code)
        except BaseException as e:
            failed_codes.append(code)
            pass
        continue
    elapsed = time.time() - start
    print('计算' + str(len(codes)) + '个股票, 共花费' + str(elapsed) + '秒')
    if failed_codes:
        print('失败股票codes=')
        print(failed_codes)
    return result_codes

        
## 主函数
def cal_main():
    start = time.time()
    localtime = time.localtime(start)
    strTime = time.strftime("%Y-%m-%d %H:%M:%S", localtime)
    print('开始时间为 ' + strTime)

    codeStr = fIO.fromText('stocks-config')
    codes = codeStr.split(',')
    global result
    global last_result
    result = fetchResult(codes)
    if result:
        last_result = list(set(result).difference(set(last_result)))
        result = []
    
    ##发邮件 用守护线程包裹，作为邮件发送超时的一种防御(但守护线程会一直结束不了)
    if last_result:
        print(last_result)
        topic = '【股票买卖点】'
        text = '以下股票出现买点/卖点\n' + str(last_result)
        t = threading.Thread(target=fmail.mail, args = (topic, text))
        t.setDaemon(True)
        t.start()
        t.join(2)
    end = time.time()
    elapsed = end - start
    localtime = time.localtime(end)
    strTime = time.strftime("%Y-%m-%d %H:%M:%S", localtime)
    print('结束时间为  ' + strTime)
    print('计算' + str(len(codes)) + '个股票, 共花费' + str(elapsed) + '秒')
    print('Exiting cal_main Thread')
print('done')

done


In [5]:
cal_main()

开始时间为 2019-08-12 21:08:30
计算3个股票, 共花费0.6515829563140869秒
['600980', '002201']
邮件发送成功
结束时间为  2019-08-12 21:08:32
计算3个股票, 共花费2.000993013381958秒
Exiting cal_main Thread


In [ ]:
from apscheduler.schedulers.background import BackgroundScheduler

scheduler = BackgroundScheduler()  
scheduler.add_job(cal_main, 'cron', minute = '1/2')
# scheduler.add_job(cal_main, 'cron', day_of_week = '0-4', hour = '10-11, 13-14', minute = '1/2')
# scheduler.add_job(cal_main, 'cron', day_of_week = '0-4', hour = '9', minute = '36/2')
scheduler.start()
print('start')